## easy_torch usage examples

#### Convolutional neural network trained on CIFAR100 Dataset loaded from torchvision:

In [1]:
import torch, torchvision
from torchvision import transforms

# import CIFAR100 Dataset
print('######### CIFAR100 ################')

transform = transforms.Compose([transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                    download=True, transform=transform)

# convert to DataLoaders
batch_size = 64
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)
print("Dataset classes : \n",trainset.classes)

######### CIFAR100 ################
Files already downloaded and verified
Files already downloaded and verified
Dataset classes : 
 ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'tab

In [2]:
# Train a convolutional network on the CIFAR100 Dataset
from easy_torch import EasyTorchConv

# create the convolutional network with residual layers: first layer is the channel number
covn = EasyTorchConv(len(trainset.classes), conv_layer_sizes=(3,256,512,512,1024,1024),
                    full_layer_sizes=(4096,), residual_inserts=(3,4,5), batch_norm=True, actv_func=torch.nn.ELU)
covn.fit(trainloader, testloader, epochs=20, lr=1e-4, weight_decay=1e-4, opt_func=torch.optim.Adam, verbose=True)

# NOTE: Easist way to figure out first layer size for full_layer_size: 
# enter the convolutional layer sizes you'd like, plus any number for
# the full layers, and get the proper full layer size from the resulting error

Network design:  EasyTorchConv(
  (net): ModuleDict(
    (layer_1): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchNorm_1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation_1): ELU(alpha=1.0, inplace=True)
    (layer_2): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchNorm_2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation_2): ELU(alpha=1.0, inplace=True)
    (maxPool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (dropout_2): Dropout(p=0.5, inplace=False)
    (layer_3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batchNorm_3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation_3): ELU(alpha=1.0, inplace=True)
    (residual_3): Sequential(
      (layer_0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

##### MLP neural network trained on randomized Dataset loaded from numpy arrays:

In [3]:
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

import dataloading
from easy_torch import EasyTorchMLP

X, y = make_blobs(n_samples=20000, n_features=100, centers=5, cluster_std=10) #, random_state=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7) #, random_state=2)
trainloader, testloader = dataloading.from_np(X_train, X_test, y_train, y_test, batch_size=64)

# data shape to find number of features and classes
num_rows, num_cols = X_train.shape
unique_y = np.unique(y_train) # i.e. classes

# initialize model, train and test
mlp = EasyTorchMLP(num_cols, len(unique_y), hidden_layer_sizes=(2048,1024,512), residual_inserts=None)
mlp.fit(trainloader, testloader, epochs=5, lr=1e-3, weight_decay=0, verbose=True)

Network design:  EasyTorchMLP(
  (net): ModuleDict(
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (layer_0): Linear(in_features=100, out_features=2048, bias=True)
    (activation_0): ReLU(inplace=True)
    (layer_1): Linear(in_features=2048, out_features=1024, bias=True)
    (activation_1): ReLU(inplace=True)
    (layer_2): Linear(in_features=1024, out_features=512, bias=True)
    (activation_2): ReLU(inplace=True)
    (output): Linear(in_features=512, out_features=5, bias=True)
  )
)
#### Start training: ####
Device: cuda
Training...
Epoch 0: Train loss: 0.855, accuracy: 0.848 :: Test loss: 0.354 accuracy: 0.998
Epoch 1: Train loss: 0.209, accuracy: 0.999 :: Test loss: 0.128 accuracy: 1.000
Epoch 2: Train loss: 0.093, accuracy: 1.000 :: Test loss: 0.072 accuracy: 1.000
Epoch 3: Train loss: 0.057, accuracy: 1.000 :: Test loss: 0.049 accuracy: 1.000
Epoch 4: Train loss: 0.040, accuracy: 1.000 :: Test loss: 0.037 accuracy: 1.000
Training finished.
Final train loss: 0.040 :: accurac

In [ ]:
"""
#(example save and load of the MLP model:)
model = mlp
torch.save(model.state_dict(), "./model/model.torch")

model.load_state_dict(torch.load("./model/model.torch"))
model.eval()
"""